In [1]:
import mediapipe as mp

In [2]:
import cv2

In [3]:
mp_drawing=mp.solutions.drawing_utils
mp_hands=mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
cap = cv2.VideoCapture("http://192.168.155.88:8080/video")
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [16]:
cap = cv2.VideoCapture("http://192.168.49.139:8080/video")
# Initiate holistic model
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while(True):
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [116]:
num_coords=len(results.multi_hand_landmarks[0].landmark)
num_coords

21

In [117]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [118]:
import csv
import os
import numpy as np

In [124]:
with open('landmarks.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [129]:
class_name="Rock"

In [130]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while(True):
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        
        try:# Extract Pose landmarks
            hands = results.multi_hand_landmarks[0].landmark
            row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hands]).flatten())
            row.insert(0, class_name)
        
        # Export to CSV
            with open('landmarks.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
        except:
            pass
        
        
            
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [90]:
len(row)

64

In [131]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('landmarks.csv')
X = df.drop('class', axis=1) 
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [132]:
df.dtypes

class     object
x1       float64
y1       float64
z1       float64
x2       float64
          ...   
y20      float64
z20      float64
x21      float64
y21      float64
z21      float64
Length: 64, dtype: object

In [133]:
import sklearn.pipeline
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [134]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
}

In [135]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [136]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())])}

In [137]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0


In [138]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [139]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [140]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [152]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while(True):
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        
        try:
            hands = results.multi_hand_landmarks[0].landmark
            row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hands]).flatten())
        
        
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            
            cv2.putText(image, body_language_class, (10,450),cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 2, cv2.LINE_AA)
            
            print(body_language_class)
        except:
            pass
       
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Paper
Paper
Rock
Rock
Scissor
Scissor
Scissor
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Scissor
Scissor
Scissor
Scissor
Paper
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Paper
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Paper
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Paper
Paper
Paper
Paper
Paper
Paper
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Rock
Rock
Rock
Rock
Rock
Rock
Scissor
Scissor
Scissor
Scissor
Scissor
Paper
Paper
Paper
Paper
Paper
Paper
Scissor
Scissor
Scissor
Scissor
Scissor
Scissor
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Scissor
Scissor
Scissor
Scissor
Sciss

In [142]:
mp_hands??